## Inverse Dynamics of a 3-Link RRR Articulated Robot Arm

In [ ]:
from sympy import *
from sympy.physics.mechanics import LagrangesMethod, dynamicsymbols
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from IPython.display import display, Markdown, Math

In [ ]:
def dispmath(lhs, ltx): display(Math(f'{lhs} = {latex(ltx)}'))

Defining the necessary symbols/functions:

In [ ]:
m1, m2, m3, l1, l2, l3, r, g, t = symbols('m1, m2, m3, l1, l2, l3, r, g, t')
th1, th2, th3 = dynamicsymbols('theta1, theta2, theta3')

Defining the positions of the centers of mass for the 2nd and 3rd links:

In [ ]:
x2 = l2 * cos(th2) * cos(th1) / 2
y2 = l2 * cos(th2) * sin(th1) / 2
z2 = l1 + l2 * sin(th2) / 2

In [ ]:
x3 = l2 * cos(th2) * cos(th1) + l3 * cos(th3) * cos(th1) / 2
y3 = l2 * cos(th2) * sin(th1) + l3 * cos(th3) * sin(th1) / 2
z3 = l1 + l2 * sin(th2) + l3 * sin(th3) / 2

In [ ]:
x2d = x2.diff(t); dispmath('\dot{x_2}', x2d)
y2d = y2.diff(t); dispmath('\dot{y_2}', y2d)
z2d = z2.diff(t); dispmath('\dot{z_2}', z2d)

In [ ]:
x3d = x3.diff(t); dispmath('\dot{x_3}', x3d)
y3d = y3.diff(t); dispmath('\dot{y_3}', y3d)
z3d = z3.diff(t); dispmath('\dot{z_3}', z3d)

Inertia matrices for the 2nd an 3rd links:

In [ ]:
I2 = Matrix([[m2*l2**2/12 + m2*r**2/4, 0, 0], [0, m2*l2**2/12 + m2*r**2/4, 0], [0, 0, m2*r**2/2]])
dispmath('I_2', I2)
I3 = Matrix([[m3*l3**2/12 + m3*r**2/4, 0, 0], [0, m3*l2**2/12 + m3*r**2/4, 0], [0, 0, m3*r**2/2]])
dispmath('I_3', I3)

Shorthands for the derivatives:

In [ ]:
th1d = Derivative(th1, t)
th1dd = Derivative(th1d, t)
th2d = Derivative(th2, t)
th2dd = Derivative(th2d, t)
th3d = Derivative(th3, t)
th3dd = Derivative(th3d, t)

Angular velocity column matrices:

In [ ]:
omega2 = Matrix([[0], [th2d], [th1d]])
dispmath('\omega_2', omega2)
omega3 = Matrix([[0], [th3d], [th1d]])
dispmath('\omega_3', omega3)

Computing the kinetic energies:

In [ ]:
Ek1 = m1 * r**2 * th1d**2 / 2
Ek2 = simplify(m2 * (x2d**2 + y2d**2 + z2d**2) / 2 + (omega2.T * I2 * omega2 / 2)[0])
Ek3 = simplify(m3 * (x3d**2 + y3d**2 + z3d**2) / 2 + (omega3.T * I3 * omega3 / 2)[0])
dispmath('E_{k1}', Ek1)
dispmath('E_{k2}', Ek2)
dispmath('E_{k3}', Ek3)

Computing the potential energies:

In [ ]:
Ep1 = m1 * g * l1 / 2
Ep2 = m2 * g * z2
Ep3 = m3 * g * z3
dispmath('E_{p1}', Ep1)
dispmath('E_{p2}', Ep2)
dispmath('E_{p3}', Ep3)

The Lagrangian function:

In [ ]:
L = Ek1 + Ek2 + Ek3 - Ep1 - Ep2 - Ep3
dispmath('L', L)

Deriving the equations of motion:

In [ ]:
LM = LagrangesMethod(L, (th1, th2, th3))
eqns = LM.form_lagranges_equations()
dispmath('\\begin{bmatrix} T_1 \\\\ T_2 \\\\ T_3 \\end{bmatrix}', simplify(eqns))

Turning the equations of motion into functions that take numerical values:

In [ ]:
params = [m1, m2, m3, l1, l2, l3, r, g, th1, th1d, th1dd, th2, th2d, th2dd, th3, th3d, th3dd]
T1_lambda = utilities.lambdify(params, eqns[0])
T2_lambda = utilities.lambdify(params, eqns[1])
T3_lambda = utilities.lambdify(params, eqns[2])

Introducing numerical values:

In [ ]:
L1 = 0.5
L2 = 0.7
L3 = 0.6
R = 0.05
M1 = 1
M2 = 1.4
M3 = 1.2
G = 9.81

The general shape of the assumed trajectory:

In [ ]:
alpha = np.linspace(0, np.pi, 1000);
x = np.linspace(0, 1, len(alpha))
r = x**2 + 1
y = r * np.cos(alpha)
z = r * np.sin(alpha)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.view_init(elev=30, azim=35)
ax.plot3D(x, y, z, 'b')
plt.show()

Fitting the trajectory into a "box" within the workspace:

In [ ]:
def fit(x, x_min, x_max):
    fitted = (x - np.min(x)) * (x_max - x_min) / (np.max(x) - np.min(x)) + x_min
    return fitted

In [ ]:
x_fit = fit(x, 0.4, 0.8)
y_fit = fit(y, 0.4, 0.8)
z_fit = fit(z, 0, 0.3)

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.view_init(elev=30, azim=35)
ax.plot3D(x_fit, y_fit, z_fit, 'g')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim3d(-0.1, 1)
ax.set_ylim3d(-0.1, 1)
ax.set_zlim3d(-0.1, 1)

plt.show()

Converting the trajectory to the joint variables:

![Two Circles](two_circles.png)

In [ ]:
s = np.sqrt(x_fit**2 + y_fit**2)
A = (L2**2 - L3**2 + s**2 + z_fit**2) / (2 * s)
B = z_fit / s

zA = L1
a = B**2 + 1
b = -2 * (A * B + zA)
c = A**2 + zA**2 - L2**2
delta = b**2 - 4 * a * c
zB = (-b + np.sqrt(delta)) / (2 * a)
sB = A - B * zB

TH1 = np.arctan2(y_fit, x_fit)
TH2 = np.arctan2(zB - zA, sB)
TH3 = -np.arctan2(zB - z_fit, s - sB)

Simulation time and time step:

In [ ]:
ts = np.linspace(0, 10, len(x))
dt = ts[1] - ts[0]

In [ ]:
def interpolate(x, num_elements):
    old_indices = np.arange(len(x))
    new_indices = np.linspace(0, len(x) - 1, num_elements)
    interpolated = np.interp(new_indices, old_indices, x)
    return interpolated

Numerical differentiantion:

In [ ]:
TH1d = np.diff(TH1) / dt
TH1dd = np.diff(TH1d) / dt
TH2d = np.diff(TH2) / dt
TH2dd = np.diff(TH2d) / dt
TH3d = np.diff(TH3) / dt
TH3dd = np.diff(TH3d) / dt

TH1d_interp = interpolate(TH1d, len(TH1))
TH1dd_interp = interpolate(TH1dd, len(TH1))
TH2d_interp = interpolate(TH2d, len(TH2))
TH2dd_interp = interpolate(TH2dd, len(TH2))
TH3d_interp = interpolate(TH3d, len(TH3))
TH3dd_interp = interpolate(TH3dd, len(TH3))

Inputting numerical values into the previously genenerated functions:

In [ ]:
num_vals = (M1, M2, M3, L1, L2, L3, R, G, TH1, TH1d_interp, TH1dd_interp, TH2, TH2d_interp, TH2dd_interp, TH3, TH3d_interp, TH3dd_interp)
T1_n = T1_lambda(*num_vals)
T2_n = T2_lambda(*num_vals)
T3_n = T3_lambda(*num_vals)

In [ ]:
plt.plot(ts, T1_n)
plt.xlabel('t [$s$]')
plt.ylabel('$T_1$ [$N \cdot m$]')
plt.grid()

In [ ]:
plt.plot(ts, T2_n)
plt.xlabel('t [$s$]')
plt.ylabel('$T_2$ [$N \cdot m$]')
plt.grid()

In [ ]:
plt.plot(ts, T3_n)
plt.xlabel('t [$s$]')
plt.ylabel('$T_3$ [$N \cdot m$]')
plt.grid()